<a href="https://colab.research.google.com/github/rcrozatti/Analises-Redes-Sociais/blob/main/Datajor_An%C3%A1lise_de_termos_dos_governadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Termos governadores Twitter e Instagram

## Importando bibliotecas

In [44]:
import numpy                  as np
import pandas                 as pd
from nltk.corpus import stopwords
from google.colab import files
from google.colab import drive
import re
from nltk.util import ngrams
drive.mount('/content/drive')
import nltk
nltk.download('stopwords')
import datetime
from datetime import datetime
import matplotlib.pyplot as plt 
import seaborn as sns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##Análise da planilha

Neste projeto, serão analisadas duas planilhas - respectivamente, do Twitter e do Instagram, dos 27 governadores dos 26 estados e do Distrito Federal. Para a raspagem, foi utilizado a ferramenta paga Phantom Buster.

### Twitter

Além de abrir a planilha referente ao Twitter, usei a biblioteca Datetime para corrigir a data e a hora da planilha original, gerando duas novas colunas - data e hora. 

In [2]:
dados_twitter = pd.read_csv('governadores_twitter_12-12-22.csv')
dados_twitter['data'] = pd.to_datetime(dados_twitter['tweetDate']).dt.date
dados_twitter['hora'] = pd.to_datetime(dados_twitter['tweetDate']).dt.time
dados_twitter = dados_twitter[['handle', 'data', 'text']]
dados_twitter.columns = ['nome', 'data', 'texto']
dados_twitter.head()

,nome,data,texto
0,GladsonCameli,2022-11-29,Que alegria ver 305 famílias do bairro Montanh...
1,GladsonCameli,2022-11-24,O HEXA VEM! 🏆\n\nBora torcer pelo Brasil na es...
2,GladsonCameli,2022-11-21,"Olá, meus amigos! \n\nNesta tarde estive na Ca..."
3,GladsonCameli,2022-11-21,Também fizemos a entrega de 9 quadriciclos par...
4,GladsonCameli,2022-11-17,Quero agradecer toda a comitiva do Acre que es...


Aqui, conferi se os nomes filtrados são os nomes corretos.

In [3]:
dados_twitter.nome.unique()

array(['GladsonCameli', 'paulodantasal', 'waldezoficial', 'wilsonlimaAM',
       'costa_rui', 'IzoldaCelaCe', 'IbaneisOficial', 'Casagrande_ES',
       'ronaldocaiado', 'carlosbrandaoma', 'Reinaldo45psdb', 'RomeuZema',
       'helderbarbalho', 'joaoazevedolins', 'ratinho_jr', 'PauloCamara40',
       'ReginaSousa_01', 'claudiocastroRJ', 'fatimabezerra',
       'DelegadoRanolfo', 'celmarcosrocha', 'antoniodenarium',
       'CarlosMoises', 'rodrigogarcia_', 'belivaldochagas', 'WanderleiTO'],
      dtype=object)

#### Limpeza e organização dos nomes

Como o dataframe criado com o Phantombuster informa somente o nome de usuário, aqui retirei os usuários do Twitter para o nome do político, para facilitar os filtros após a junção com a planilha do Instagram. Também criei uma nova coluna, identificando a rede social.

In [4]:
dados_tw = dados_twitter.replace({'GladsonCameli': 'Gladson Cameli', 
                  'paulodantasal': 'Paulo Dantas', 
                  'waldezoficial': 'Waldez Góes', 
                  'wilsonlimaAM': 'Wilson Lima', 
                  'costa_rui': 'Rui Costa', 
                  'IzoldaCelaCe': 'Izolda Cela', 
                  'IbaneisOficial': 'Ibaneis Rocha', 
                  'Casagrande_ES': 'Renato Casagrande', 
                  'ronaldocaiado': 'Ronaldo Caiado', 
                  'carlosbrandaoma': 'Carlos Brandão Junior', 
                  'Reinaldo45psdb': 'Reinaldo Azambuja', 
                  'RomeuZema': 'Romeu Zema', 
                  'helderbarbalho': 'Helder Barbalho', 
                  'joaoazevedolins': 'João Azevêdo', 
                  'ratinho_jr': 'Ratinho Júnior', 
                  'PauloCamara40': 'Paulo Câmara', 
                  'ReginaSousa_01': 'Regina Sousa', 
                  'claudiocastroRJ': 'Cláudio Castro', 
                  'fatimabezerra': 'Fátima Bezerra', 
                  'DelegadoRanolfo': 'Ranolfo Vieira Júnior', 
                  'celmarcosrocha': 'Coronel Marcos Rocha', 
                  'antoniodenarium': 'Antonio Denarium', 
                  'CarlosMoises': 'Carlos Moisés', 
                  'rodrigogarcia_': 'Rodrigo Garcia', 
                  'belivaldochagas': 'Belivaldo Chagas', 
                  'WanderleiTO': 'Wanderlei Barbosa'})
dados_tw['rede social'] = 'Twitter'
dados_tw.head()

,nome,data,texto,rede social
0,Gladson Cameli,2022-11-29,Que alegria ver 305 famílias do bairro Montanh...,Twitter
1,Gladson Cameli,2022-11-24,O HEXA VEM! 🏆\n\nBora torcer pelo Brasil na es...,Twitter
2,Gladson Cameli,2022-11-21,"Olá, meus amigos! \n\nNesta tarde estive na Ca...",Twitter
3,Gladson Cameli,2022-11-21,Também fizemos a entrega de 9 quadriciclos par...,Twitter
4,Gladson Cameli,2022-11-17,Quero agradecer toda a comitiva do Acre que es...,Twitter


No Twitter são 803 postagens raspadas.

In [5]:
dados_tw.shape[0]

803

### Instagram

Vamos repetir o mesmo processo com o Instagram antes de juntar as duas bases de dados.

In [6]:

dados_instagram = pd.read_csv('governadores_insta.csv')
dados_instagram['data'] = pd.to_datetime(dados_instagram['pubDate']).dt.date
dados_instagram['hora'] = pd.to_datetime(dados_instagram['pubDate']).dt.time
dados_instagram = dados_instagram[['profileUrl', 'data', 'description']]
dados_instagram.columns = ['nome', 'data', 'texto']
dados_instagram.head()

,nome,data,texto
0,https://www.instagram.com/gladsoncameli,2022-12-11,Quero parabenizar meu primo @marmude.camely qu...
1,https://www.instagram.com/gladsoncameli,2022-12-11,Prestigiei também a confraternização da Associ...
2,https://www.instagram.com/gladsoncameli,2022-12-11,Luzes do Natal! 🎅\n\nO espírito natalino invad...
3,https://www.instagram.com/gladsoncameli,2022-12-11,"Então é Natal 🎅 🎄\n\nParticipei, na noite de h..."
4,https://www.instagram.com/gladsoncameli,2022-12-10,"Hoje quero felicitar a minha vice eleita, sena..."


In [7]:
dados_insta = dados_instagram.replace({'https://www.instagram.com/gladsoncameli': 'Gladson Cameli', 
                              'https://www.instagram.com/paulodantasalagoas': 'Paulo Dantas', 
                              'https://www.instagram.com/waldezgoesoficial': 'Waldez Góes', 
                              'https://www.instagram.com/wilsonlimaam': 'Wilson Lima', 
                              'https://www.instagram.com/ruicostaoficial': 'Rui Costa', 
                              'https://www.instagram.com/izoldacelace': 'Izolda Cela', 
                              'https://www.instagram.com/ibaneisoficial': 'Ibaneis Rocha', 
                              'https://www.instagram.com/casagrande_es': 'Renato Casagrande', 
                              'https://www.instagram.com/ronaldocaiado': 'Ronaldo Caiado', 
                              'https://www.instagram.com/carlosbrandaoma': 'Carlos Brandão Junior', 
                              'https://www.instagram.com/mauromendesoficial': 'Mauro Mendes', 
                              'https://www.instagram.com/reinaldoazambuja': 'Reinaldo Azambuja', 
                              'https://www.instagram.com/romeuzemaoficial': 'Romeu Zema', 
                              'https://www.instagram.com/helderbarbalho': 'Helder Barbalho', 
                              'https://www.instagram.com/joaoazevedolins': 'João Azevêdo', 
                              'https://www.instagram.com/ratinho_junior': 'Ratinho Júnior', 
                              'https://www.instagram.com/paulocamara40': 'Paulo Câmara', 
                              'https://www.instagram.com/senadora_regina_sousa': 'Regina Sousa', 
                              'https://www.instagram.com/claudiocastrorj': 'Cláudio Castro', 
                              'https://www.instagram.com/fatimabezerra13': 'Fátima Bezerra', 
                              'https://www.instagram.com/delegadoranolfo': 'Ranolfo Vieira Júnior', 
                              'https://www.instagram.com/celmarcosrocha': 'Coronel Marcos Rocha', 
                              'https://www.instagram.com/antoniodenariumrr': 'Antonio Denarium', 
                              'https://www.instagram.com/governadormoises': 'Carlos Moisés', 
                              'https://www.instagram.com/rodrigogarciaoficial': 'Rodrigo Garcia', 
                              'https://www.instagram.com/belivaldochagas': 'Belivaldo Chagas', 
                              'https://www.instagram.com/wanderleibarbosaoficial': 'Wanderlei Barbosa'})
dados_insta['rede social'] = 'Instagram'
dados_insta.head()

,nome,data,texto,rede social
0,Gladson Cameli,2022-12-11,Quero parabenizar meu primo @marmude.camely qu...,Instagram
1,Gladson Cameli,2022-12-11,Prestigiei também a confraternização da Associ...,Instagram
2,Gladson Cameli,2022-12-11,Luzes do Natal! 🎅\n\nO espírito natalino invad...,Instagram
3,Gladson Cameli,2022-12-11,"Então é Natal 🎅 🎄\n\nParticipei, na noite de h...",Instagram
4,Gladson Cameli,2022-12-10,"Hoje quero felicitar a minha vice eleita, sena...",Instagram


Uma diferença da planilha do Instagram em relação ao Twitter é que, nesse caso, a raspagem pegou publicações de perfis que fez postagens em conjunto com a nossa lista de governadores. Mais abaixo faremos um filtro para selecionar somente os governadores. 

In [8]:
dados_insta.nome.unique()

array(['Gladson Cameli', 'https://www.instagram.com/progressistas',
       'https://www.instagram.com/governo.acre', 'Paulo Dantas',
       'Waldez Góes', 'Wilson Lima', 'Rui Costa',
       'https://www.instagram.com/govba', 'Izolda Cela', 'Ibaneis Rocha',
       'Renato Casagrande', 'Ronaldo Caiado', 'Carlos Brandão Junior',
       'https://www.instagram.com/virginiamendesoficial', 'Mauro Mendes',
       'https://www.instagram.com/uniaobrasil44', 'Reinaldo Azambuja',
       'Romeu Zema', 'Helder Barbalho', 'João Azevêdo', 'Ratinho Júnior',
       'Paulo Câmara', 'Regina Sousa', 'Cláudio Castro', 'Fátima Bezerra',
       'Ranolfo Vieira Júnior', 'Coronel Marcos Rocha',
       'https://www.instagram.com/luanarochaoficialro',
       'Antonio Denarium', 'https://www.instagram.com/simonedenarium',
       'Carlos Moisés', 'Rodrigo Garcia', 'Belivaldo Chagas', nan],
      dtype=object)

##Concatenar as duas bases de dados

In [9]:
dados_social = pd.concat([dados_tw, dados_insta])
dados_social.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1296 entries, 0 to 492
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nome         1295 non-null   object
 1   data         1295 non-null   object
 2   texto        1288 non-null   object
 3   rede social  1296 non-null   object
dtypes: object(4)
memory usage: 50.6+ KB


## Limpar dados nulos

In [10]:
dados_social[dados_social['texto'].isnull()]

,nome,data,texto,rede social
45,Wilson Lima,2022-12-05,NaN,Instagram
103,Ronaldo Caiado,2022-12-06,NaN,Instagram
106,Ronaldo Caiado,2022-12-04,NaN,Instagram
343,Regina Sousa,2019-01-03,NaN,Instagram
347,Regina Sousa,2019-01-02,NaN,Instagram
351,Regina Sousa,2018-12-21,NaN,Instagram
466,Carlos Moisés,2022-12-01,NaN,Instagram
492,NaN,NaT,NaN,Instagram


In [11]:
dados_social.dropna(subset = ['texto'], inplace = True)
dados_social.index = range(dados_social.shape[0])
dados_social.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1288 entries, 0 to 1287
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nome         1288 non-null   object
 1   data         1288 non-null   object
 2   texto        1288 non-null   object
 3   rede social  1288 non-null   object
dtypes: object(4)
memory usage: 40.4+ KB


## Selecionar a data para análise

A raspagem foi feita pela última vez no dia 12/12/2022. Por isso, vamos selecionar os quatro anos anteriores, desde quando começou à última legislatura. 

In [12]:
s = pd.to_datetime(dados_social['data'])
assert str(s.dtype) == 'datetime64[ns]'
dados_social.index = s
print(dados_social['2019-01-01':'2022-12-12'])
dados_data = dados_social['2019-01-01':'2022-12-12']

                        nome        data  \
data                                       
2022-11-29    Gladson Cameli  2022-11-29   
2022-11-24    Gladson Cameli  2022-11-24   
2022-11-21    Gladson Cameli  2022-11-21   
2022-11-21    Gladson Cameli  2022-11-21   
2022-11-17    Gladson Cameli  2022-11-17   
...                      ...         ...   
2022-12-07  Belivaldo Chagas  2022-12-07   
2022-12-07  Belivaldo Chagas  2022-12-07   
2022-12-07  Belivaldo Chagas  2022-12-07   
2022-12-07  Belivaldo Chagas  2022-12-07   
2022-12-06  Belivaldo Chagas  2022-12-06   

                                                        texto rede social  
data                                                                       
2022-11-29  Que alegria ver 305 famílias do bairro Montanh...     Twitter  
2022-11-24  O HEXA VEM! 🏆\n\nBora torcer pelo Brasil na es...     Twitter  
2022-11-21  Olá, meus amigos! \n\nNesta tarde estive na Ca...     Twitter  
2022-11-21  Também fizemos a entrega de 9 quadr

Conferir se as datas filtradas estão corretas.

##Filtrar somente os governadores

Aqui criei uma lista com os nomes dos governadores da legislatura 2019-2022 para fazer um filtro.

In [14]:
lista_governadores = ['Gladson Cameli', 'Paulo Dantas', 'Waldez Góes', 'Wilson Lima', 'Rui Costa', 'Izolda Cela', 
                      'Ibaneis Rocha', 'Renato Casagrande', 'Ronaldo Caiado', 'Carlos Brandão Junior', 'Mauro Mendes', 
                      'Reinaldo Azambuja', 'Romeu Zema', 'Helder Barbalho', 'João Azevêdo', 'Ratinho Júnior', 
                      'Paulo Câmara', 'Regina Sousa', 'Cláudio Castro', 'Fátima Bezerra', 'Ranolfo Vieira Júnior', 
                      'Coronel Marcos Rocha', 'Antonio Denarium', 'Carlos Moisés', 'Rodrigo Garcia', 'Belivaldo Chagas', 
                      'Wanderlei Barbosa']
lista_governadores

['Gladson Cameli',
 'Paulo Dantas',
 'Waldez Góes',
 'Wilson Lima',
 'Rui Costa',
 'Izolda Cela',
 'Ibaneis Rocha',
 'Renato Casagrande',
 'Ronaldo Caiado',
 'Carlos Brandão Junior',
 'Mauro Mendes',
 'Reinaldo Azambuja',
 'Romeu Zema',
 'Helder Barbalho',
 'João Azevêdo',
 'Ratinho Júnior',
 'Paulo Câmara',
 'Regina Sousa',
 'Cláudio Castro',
 'Fátima Bezerra',
 'Ranolfo Vieira Júnior',
 'Coronel Marcos Rocha',
 'Antonio Denarium',
 'Carlos Moisés',
 'Rodrigo Garcia',
 'Belivaldo Chagas',
 'Wanderlei Barbosa']

In [15]:
gov_df = dados_data.query('nome in @lista_governadores')
list(gov_df['nome'].drop_duplicates())

['Gladson Cameli',
 'Paulo Dantas',
 'Waldez Góes',
 'Wilson Lima',
 'Rui Costa',
 'Izolda Cela',
 'Ibaneis Rocha',
 'Renato Casagrande',
 'Ronaldo Caiado',
 'Carlos Brandão Junior',
 'Reinaldo Azambuja',
 'Romeu Zema',
 'Helder Barbalho',
 'João Azevêdo',
 'Ratinho Júnior',
 'Paulo Câmara',
 'Regina Sousa',
 'Cláudio Castro',
 'Fátima Bezerra',
 'Ranolfo Vieira Júnior',
 'Coronel Marcos Rocha',
 'Antonio Denarium',
 'Carlos Moisés',
 'Rodrigo Garcia',
 'Belivaldo Chagas',
 'Wanderlei Barbosa',
 'Mauro Mendes']

In [16]:
gov_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1270 entries, 2022-11-29 to 2022-12-06
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nome         1270 non-null   object
 1   data         1270 non-null   object
 2   texto        1270 non-null   object
 3   rede social  1270 non-null   object
dtypes: object(4)
memory usage: 49.6+ KB


In [17]:
gov_df.head()

,nome,data,texto,rede social
data,,,,
2022-11-29,Gladson Cameli,2022-11-29,Que alegria ver 305 famílias do bairro Montanh...,Twitter
2022-11-24,Gladson Cameli,2022-11-24,O HEXA VEM! 🏆\n\nBora torcer pelo Brasil na es...,Twitter
2022-11-21,Gladson Cameli,2022-11-21,"Olá, meus amigos! \n\nNesta tarde estive na Ca...",Twitter
2022-11-21,Gladson Cameli,2022-11-21,Também fizemos a entrega de 9 quadriciclos par...,Twitter
2022-11-17,Gladson Cameli,2022-11-17,Quero agradecer toda a comitiva do Acre que es...,Twitter


## Limpeza


Antes de filtrar os termos, vamos limpar a planilha usando algumas funções importantes: letras das postagens em caixa baixa e *stop words* - biblioteca para deletar palavras comuns, como artigos ou termos indesejados para a análise.

In [18]:
#caixa baixa e limpeza de caracteres especiais
gov_df['texto'] = gov_df['texto'].str.lower().map(lambda x: re.sub(r'[^\w\s]', '', x))
gov_df['postagens'] = gov_df['texto'].str.replace('\d+', '')
gov_df.head()

<ipython-input-18-ed16c0ae51be>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gov_df['texto'] = gov_df['texto'].str.lower().map(lambda x: re.sub(r'[^\w\s]', '', x))
<ipython-input-18-ed16c0ae51be>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  gov_df['postagens'] = gov_df['texto'].str.replace('\d+', '')
<ipython-input-18-ed16c0ae51be>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gov_df['postagens'] = gov_df['texto'].str.replace('\d+', '')


,nome,data,texto,rede social,postagens
data,,,,,
2022-11-29,Gladson Cameli,2022-11-29,que alegria ver 305 famílias do bairro montanh...,Twitter,que alegria ver famílias do bairro montanhês ...
2022-11-24,Gladson Cameli,2022-11-24,o hexa vem \n\nbora torcer pelo brasil na estr...,Twitter,o hexa vem \n\nbora torcer pelo brasil na estr...
2022-11-21,Gladson Cameli,2022-11-21,olá meus amigos \n\nnesta tarde estive na casa...,Twitter,olá meus amigos \n\nnesta tarde estive na casa...
2022-11-21,Gladson Cameli,2022-11-21,também fizemos a entrega de 9 quadriciclos par...,Twitter,também fizemos a entrega de quadriciclos para...
2022-11-17,Gladson Cameli,2022-11-17,quero agradecer toda a comitiva do acre que es...,Twitter,quero agradecer toda a comitiva do acre que es...


In [19]:
#stop words
stop_words = '/content/drive/MyDrive/lista_stopwords.txt'
stop = stopwords.words('portuguese')
with open(stop_words, "r") as f:
    for line in f:
        stop.extend(line.split()) 

In [20]:
gov_df['postagens'] = gov_df['postagens'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
gov_df.head()

<ipython-input-20-816148612d4c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gov_df['postagens'] = gov_df['postagens'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


,nome,data,texto,rede social,postagens
data,,,,,
2022-11-29,Gladson Cameli,2022-11-29,que alegria ver 305 famílias do bairro montanh...,Twitter,alegria ver famílias bairro montanhês branco r...
2022-11-24,Gladson Cameli,2022-11-24,o hexa vem \n\nbora torcer pelo brasil na estr...,Twitter,hexa bora torcer estreia seleção brasileira h ...
2022-11-21,Gladson Cameli,2022-11-21,olá meus amigos \n\nnesta tarde estive na casa...,Twitter,olá equipe realizando reunião técnica transiçã...
2022-11-21,Gladson Cameli,2022-11-21,também fizemos a entrega de 9 quadriciclos par...,Twitter,fizemos entrega quadriciclos corpo bombeiros a...
2022-11-17,Gladson Cameli,2022-11-17,quero agradecer toda a comitiva do acre que es...,Twitter,agradecer toda comitiva acre junto comigo cope...


## Separar palavras

Parte importante da análise de texto: separar todas as palavras e contabilizar as que mais se repetem. Aqui utilizei a NGram, da biblioteca NLTK. Com ela, é possível selecionar a quantidade de termos que aparecem juntos para contabilizar. É possível fazer análises com mais de um termo junto, mas nesse caso foi somente com um.

In [21]:
n_1 = 1
gov_termos_1 = gov_df
gov_termos_1['termos'] = gov_df['postagens'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n_1))))
gov_termos_1 = (gov_termos_1.assign(count=gov_termos_1['termos'].str.len()).explode('termos').query('count > 0'))
gov_termos_1 = gov_termos_1.drop(columns=['count'])
gov_termos_1.head()

<ipython-input-21-a2376ecc9719>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gov_termos_1['termos'] = gov_df['postagens'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n_1))))


,nome,data,texto,rede social,postagens,termos
data,,,,,,
2022-11-29,Gladson Cameli,2022-11-29,que alegria ver 305 famílias do bairro montanh...,Twitter,alegria ver famílias bairro montanhês branco r...,alegria
2022-11-29,Gladson Cameli,2022-11-29,que alegria ver 305 famílias do bairro montanh...,Twitter,alegria ver famílias bairro montanhês branco r...,ver
2022-11-29,Gladson Cameli,2022-11-29,que alegria ver 305 famílias do bairro montanh...,Twitter,alegria ver famílias bairro montanhês branco r...,famílias
2022-11-29,Gladson Cameli,2022-11-29,que alegria ver 305 famílias do bairro montanh...,Twitter,alegria ver famílias bairro montanhês branco r...,bairro
2022-11-29,Gladson Cameli,2022-11-29,que alegria ver 305 famílias do bairro montanh...,Twitter,alegria ver famílias bairro montanhês branco r...,montanhês


In [22]:
gov_termos_1.head()

,nome,data,texto,rede social,postagens,termos
data,,,,,,
2022-11-29,Gladson Cameli,2022-11-29,que alegria ver 305 famílias do bairro montanh...,Twitter,alegria ver famílias bairro montanhês branco r...,alegria
2022-11-29,Gladson Cameli,2022-11-29,que alegria ver 305 famílias do bairro montanh...,Twitter,alegria ver famílias bairro montanhês branco r...,ver
2022-11-29,Gladson Cameli,2022-11-29,que alegria ver 305 famílias do bairro montanh...,Twitter,alegria ver famílias bairro montanhês branco r...,famílias
2022-11-29,Gladson Cameli,2022-11-29,que alegria ver 305 famílias do bairro montanh...,Twitter,alegria ver famílias bairro montanhês branco r...,bairro
2022-11-29,Gladson Cameli,2022-11-29,que alegria ver 305 famílias do bairro montanh...,Twitter,alegria ver famílias bairro montanhês branco r...,montanhês


In [23]:
gov_termos_1.shape[0]

25014

In [24]:
gov_lista = gov_df.postagens.str.split(expand=True).stack().value_counts()
gov_lista

segurança          111
deus                77
população           73
desenvolvimento     71
pública             63
                  ... 
bacellar             1
renomada             1
palestra             1
presidida            1
lourivânia           1
Length: 8098, dtype: int64

In [25]:
df_termos = pd.DataFrame(gov_lista)
df_termos = gov_lista.reset_index(level=0)
df_termos.columns = ['termos', 'total']
df_termos.head(60)

,termos,total
0,segurança,111
1,deus,77
2,população,73
3,desenvolvimento,71
4,pública,63
5,educação,56
6,saúde,55
7,famílias,53
8,obras,51
9,servidores,48


##Lista com principais termos

Após analisar o dataframe com os principais termos usados nas redes sociais dos governadores, selecionei termos interessantes no debate público, além de alguns relacionados à religião para fazer um comparativo.

In [51]:
termos_lista = ['saúde',' obra','mobilidade','moradia','segurança','emprego', 'asfalto',
                'investimento','econômica', 'esporte','iluminação','escola','transporte',
                'icms','habitacional','economia','hospital', 'hospitais', 'vacina', 'habitação','educação',
                'construção','creche','inovação', 'democracia', 'violência', 'esgoto',
                'fome', 'alimento', 'comida', 'universidade', 'arma','combustível', 'cultura', 'saneamento', 
                'democracia', 'corrupção', 'renda', 'deus', 'jesus', 'natureza']

In [52]:
post_govs = gov_termos_1[gov_termos_1['termos'].isin(termos_lista)]
post_govs = post_govs.drop_duplicates()
post_govs

,nome,data,texto,rede social,postagens,termos
data,,,,,,
2022-11-21,Gladson Cameli,2022-11-21,também fizemos a entrega de 9 quadriciclos par...,Twitter,fizemos entrega quadriciclos corpo bombeiros a...,segurança
2022-11-17,Gladson Cameli,2022-11-17,quero agradecer toda a comitiva do acre que es...,Twitter,agradecer toda comitiva acre junto comigo cope...,emprego
2022-11-17,Gladson Cameli,2022-11-17,quero agradecer toda a comitiva do acre que es...,Twitter,agradecer toda comitiva acre junto comigo cope...,renda
2022-11-16,Gladson Cameli,2022-11-16,gratidão a deus pela oportunidade de represent...,Twitter,gratidão deus oportunidade representar acre co...,deus
2022-11-16,Gladson Cameli,2022-11-16,gratidão a deus pela oportunidade de represent...,Twitter,gratidão deus oportunidade representar acre co...,emprego
...,...,...,...,...,...,...
2022-12-07,Belivaldo Chagas,2022-12-07,novo centro de referência no atendimento integ...,Instagram,centro referência atendimento integral infanto...,saúde
2022-12-07,Belivaldo Chagas,2022-12-07,novo centro de referência no atendimento integ...,Instagram,centro referência atendimento integral infanto...,segurança
2022-12-06,Belivaldo Chagas,2022-12-06,mais proteção e acolhimento para as crianças e...,Instagram,proteção acolhimento crianças adolescentes vít...,violência


## Lista de estados

Para facilitar filtros posteriores, criei um dataframe para juntar com o original, contendo: governador, estado, sigla e região. 

In [53]:
estados = pd.DataFrame([['Gladson Cameli', 'Acre'], ['Paulo Dantas', 'Alagoas'], ['Waldez Góes', 'Amapá'], 
                           ['Wilson Lima', 'Amazonas'], ['Rui Costa', 'Bahia'], ['Izolda Cela', 'Ceará'], 
                           ['Ibaneis Rocha', 'Distrito Federal'], ['Renato Casagrande', 'Espírito Santo'], 
                           ['Ronaldo Caiado', 'Goiás'], ['Carlos Brandão Junior', 'Maranhão'], ['Mauro Mendes', 'Mato Grosso'], 
                           ['Reinaldo Azambuja', 'Mato Grosso do Sul'], ['Romeu Zema', 'Minas Gerais'], ['Helder Barbalho', 'Pará'], 
                           ['João Azevêdo', 'Paraíba'], ['Ratinho Júnior', 'Paraná'], ['Paulo Câmara', 'Pernambuco'], 
                           ['Regina Sousa', 'Piauí'], ['Cláudio Castro', 'Rio de Janeiro'], ['Fátima Bezerra', 'Rio Grande do Norte'], 
                           ['Ranolfo Vieira Júnior', 'Rio Grande do Sul'], ['Coronel Marcos Rocha', 'Rondônia'], ['Antonio Denarium', 'Roraima'], 
                           ['Carlos Moisés', 'Santa Catarina'], ['Rodrigo Garcia', 'São Paulo'], ['Belivaldo Chagas', 'Sergipe'], 
                           ['Wanderlei Barbosa', 'Tocantins'],],
                        columns=['nome', 'estado'])
siglas_df = pd.DataFrame([['Acre', 'AC', 'Norte'], ['Alagoas', 'AL', 'Nordeste'], ['Amapá', 'AP', 'Norte'], ['Amazonas', 'AM', 'Norte'], 
                          ['Bahia', 'BA', 'Nordeste'], ['Ceará', 'CE', 'Nordeste'], ['Distrito Federal', 'DF', 'Centro-Oeste'], 
                          ['Espírito Santo', 'ES', 'Sudeste'], ['Goiás', 'GO', 'Centro-Oeste'], ['Maranhão', 'MA', 'Nordeste'], 
                          ['Mato Grosso', 'MT', 'Centro-Oeste'], ['Mato Grosso do Sul', 'MS', 'Centro-Oeste'], ['Minas Gerais', 'MG', 'Sudeste'],
                          ['Pará', 'PA', 'Norte'], ['Paraíba', 'PB', 'Nordeste'], ['Paraná', 'PR', 'Sul'], ['Pernambuco', 'PE', 'Nordeste'], 
                          ['Piauí', 'PI', 'Nordeste'], ['Rio de Janeiro', 'RJ', 'Sudeste'], ['Rio Grande do Norte', 'RN', 'Nordeste'], 
                          ['Rio Grande do Sul', 'RS', 'Sul'], ['Rondônia', 'RO', 'Norte'], ['Roraima', 'RR', 'Norte'], ['Santa Catarina', 'SC', 'Sul'],
                          ['São Paulo', 'SP', 'Sudeste'], ['Sergipe', 'SE', 'Nordeste'], ['Tocantins', 'TO', 'Norte'],],
                        columns=['estado', 'sigla', 'região'])
estados_df = pd.merge(estados, siglas_df, on='estado')
estados_df.head()

,nome,estado,sigla,região
0,Gladson Cameli,Acre,AC,Norte
1,Paulo Dantas,Alagoas,AL,Nordeste
2,Waldez Góes,Amapá,AP,Norte
3,Wilson Lima,Amazonas,AM,Norte
4,Rui Costa,Bahia,BA,Nordeste


### Juntar as duas tabelas

In [54]:
df_governadores = pd.merge(post_govs, estados_df, on='nome')
df_governadores.head()

,nome,data,texto,rede social,postagens,termos,estado,sigla,região
0,Gladson Cameli,2022-11-21,também fizemos a entrega de 9 quadriciclos par...,Twitter,fizemos entrega quadriciclos corpo bombeiros a...,segurança,Acre,AC,Norte
1,Gladson Cameli,2022-11-17,quero agradecer toda a comitiva do acre que es...,Twitter,agradecer toda comitiva acre junto comigo cope...,emprego,Acre,AC,Norte
2,Gladson Cameli,2022-11-17,quero agradecer toda a comitiva do acre que es...,Twitter,agradecer toda comitiva acre junto comigo cope...,renda,Acre,AC,Norte
3,Gladson Cameli,2022-11-16,gratidão a deus pela oportunidade de represent...,Twitter,gratidão deus oportunidade representar acre co...,deus,Acre,AC,Norte
4,Gladson Cameli,2022-11-16,gratidão a deus pela oportunidade de represent...,Twitter,gratidão deus oportunidade representar acre co...,emprego,Acre,AC,Norte


In [55]:
df_governadores.groupby('termos').count()

,nome,data,texto,rede social,postagens,estado,sigla,região
termos,,,,,,,,
alimento,2,2,2,2,2,2,2,2
asfalto,12,12,12,12,12,12,12,12
combustível,2,2,2,2,2,2,2,2
comida,6,6,6,6,6,6,6,6
construção,20,20,20,20,20,20,20,20
corrupção,8,8,8,8,8,8,8,8
creche,2,2,2,2,2,2,2,2
cultura,21,21,21,21,21,21,21,21
democracia,23,23,23,23,23,23,23,23


# Visualização

In [56]:
# a finalizar

## Download da planilha com as postagens filtradas

In [ ]:
df_governadores.to_csv('temas_governadores.csv', index=False)
files.download('temas_governadores.csv')